Сайт: https://www.bede.net/misc/dublin/parse.html

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import stringгло
import os
from tqdm import tqdm

Сравнение результатов, которые он выдаёт, когда ищешь одно слово и когда ищешь предложение

In [2]:
a = "PRP ACCDAT 5:N S M NOM S 2:N S M ACC S 2:N S N NOM S 2:N S N ACC S 2:N S N NOM PL 2:N S N ACC PL 2:ADJ S M NOM S 2:ADJ S N NOM S 2:ADJ S N ACC S 2:V 1 P PRT IND 4:V 2 P PRT IND 4:V 3 P PRT IND 4:"
b = "PRP ACCDAT 5:N S M NOM S 2:N S M ACC S 2:N S N NOM S 2:N S N ACC S 2:N S N NOM PL 2:N S N ACC PL 2:ADJ S M NOM S 2:ADJ S N NOM S 2:ADJ S N ACC S 2:V 1 P PRT IND 4:V 2 P PRT IND 4:V 3 P PRT IND 4:"
print(a==b)

True


То есть при анализе слов он не опирается на контекст

Функция, которая отправляет запрос в форму:

In [3]:
def analyze(word):
    url = 'https://www.bede.net/cgi-bin/parserDublin/parse_dublin.cgi'
    data = {'sen': word}
    html_content = requests.post(url, data).text
    soup = BeautifulSoup(html_content, 'html.parser')
    
    word = soup.find('td', {'valign': 'top'}).text
    a = word.split('\n')
    word = a[0]
    options = a[1].split(':')[:-1]
    
    final = soup.find('td', {'bgcolor': '#ffff80'}).text
    word_par = {'word': word,
                'options': options,
                'final': final}
    return word_par

ana_dict = analyze('us')

Проверка:

In [4]:
ana_dict

{'word': 'us',
 'options': ['PRN PERS 1 ACC PL 5',
  'PRN PERS 1 DAT PL 5',
  'N S M NOM S 2',
  'N S M ACC S 2',
  'N S N NOM S 2',
  'N S N ACC S 2',
  'N S N NOM PL 2',
  'N S N ACC PL 2',
  'ADJ S M NOM S 2',
  'ADJ S N NOM S 2',
  'ADJ S N ACC S 2'],
 'final': ' PRN PERS 1 DAT PL   '}

In [5]:
analyze('we')

{'word': 'we',
 'options': ['PRN PERS 1 NOM PL 5'],
 'final': ' PRN PERS 1 NOM PL   '}

Перевод в формат через запятую, очистка глосс

In [6]:
def form_new_ana(ana):
    if '5N' in ana:
        return 'NUM'
    else:
        ana = ana[:-2]
    ana = ana.replace(' P ', ' PL ')
    ana = ana.replace('UNDECL', 'UNDCL')
    ana = ana.replace('ACCPL', 'ACC PL')
    ana = re.sub('(\d+)([SP])', r'\1 \2', ana)
    ana_list = ana.split(' ')
    if ana_list[0] == 'N':
        ana_list[0] = 'NOUN'
    if (ana_list[0] == 'NOUN' or ana_list[0] == 'ADJ') and ana_list[1] == 'S':
        ana_list[1] = 'STR'
    new_ana = ','.join(ana_list)
    return new_ana

Проверка

In [7]:
print(form_new_ana('N S M NOM S 2'))
print(form_new_ana('V 1 S PRES IND 2'))
print(form_new_ana('V MODAL 1P 5'))

NOUN,STR,M,NOM,S
V,1,S,PRES,IND
V,MODAL,1,P


Перевод в формат xml_rnc (для слова)

In [8]:
def convert_word(ana_dict):
    ana_parts = []
    for ana in ana_dict['options']:
        new_ana = form_new_ana(ana)
        ana_parts.append('<ana gr=\"'+new_ana+'\"></ana>')
    conv_form = '<w>' + ''.join(ana_parts) + ana_dict['word'] + '</w>'
    return conv_form

Проверка

In [9]:
convert_word(ana_dict)

'<w><ana gr="PRN,PERS,1,ACC,PL"></ana><ana gr="PRN,PERS,1,DAT,PL"></ana><ana gr="NOUN,STR,M,NOM,S"></ana><ana gr="NOUN,STR,M,ACC,S"></ana><ana gr="NOUN,STR,N,NOM,S"></ana><ana gr="NOUN,STR,N,ACC,S"></ana><ana gr="NOUN,STR,N,NOM,PL"></ana><ana gr="NOUN,STR,N,ACC,PL"></ana><ana gr="ADJ,STR,M,NOM,S"></ana><ana gr="ADJ,STR,N,NOM,S"></ana><ana gr="ADJ,STR,N,ACC,S"></ana>us</w>'

Перевод в формат xml_rnc (для всего текста)

In [10]:
def xml_rnc(filename):
    with open(filename, 'r', encoding='utf-8') as f, open(filename[:-4] + '.xml', 'w', encoding='utf-8') as a:
        text = f.readlines()
        new_text = '<html><head></head><body><p>'
        for line in text:
            line = line.replace('\n', '')
            if line != '':
                if line.isdigit():
                    new_line = '</p><p>'
                    print(new_line)
                else:
                    new_line = re.sub('(\w+)', lambda x: convert_word(analyze(x.group())), line)
                    new_line = '<se>'+new_line+'</se>'
                    #new_line = re.sub('^\d+$', r'</p><p>', new_line)
                    new_text += new_line
        new_text += '</p></body></html>'
        a.write(new_text)

Проверка

In [11]:
#xml_rnc('Soul and Body I.txt')

Цикл по файлам в папке, конвертирование в нужный формат

In [12]:
start_path = "."
for filename in tqdm(sorted(os.listdir(start_path))):
    if filename.endswith('.txt'):
        xml_rnc(filename)

100%|██████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<?, ?it/s]
